<a href="https://colab.research.google.com/github/TamBui1706/DeepLearningCourse/blob/main/Lab01/lab01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical

In [ ]:
# Tải dữ liệu MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Chuẩn bị dữ liệu
x_train = x_train.reshape(-1, 784) / 255.0  # Chuyển thành vector 784 chiều và chuẩn hóa [0,1]
x_test = x_test.reshape(-1, 784) / 255.0
y_train = to_categorical(y_train, 10)  # Chuyển nhãn thành one-hot encoding
y_test = to_categorical(y_test, 10)

print("Kích thước tập huấn luyện:", x_train.shape)
print("Kích thước tập kiểm tra:", x_test.shape)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Kích thước tập huấn luyện: (60000, 784)
Kích thước tập kiểm tra: (10000, 784)


In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Tránh tràn số
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

In [ ]:
def cross_entropy_loss(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred + 1e-8)) / y_true.shape[0]  # Thêm 1e-8 để tránh log(0)

In [ ]:
def initialize_weights(input_size, hidden_size, output_size):
    W1 = np.random.randn(input_size, hidden_size) * 0.01  # Trọng số lớp đầu vào -> lớp ẩn
    b1 = np.zeros((1, hidden_size))  # Bias lớp ẩn
    W2 = np.random.randn(hidden_size, output_size) * 0.01  # Trọng số lớp ẩn -> lớp đầu ra
    b2 = np.zeros((1, output_size))  # Bias lớp đầu ra
    return W1, b1, W2, b2

In [ ]:
def forward_propagation(X, W1, b1, W2, b2, activation):
    Z1 = np.dot(X, W1) + b1  # Tính toán lớp ẩn
    if activation == 'relu':
        A1 = relu(Z1)
    else:
        A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2  # Tính toán lớp đầu ra
    A2 = softmax(Z2)  # Đầu ra xác suất
    return Z1, A1, Z2, A2

In [ ]:
def backward_propagation(X, y, Z1, A1, Z2, A2, W1, W2, activation):
    m = X.shape[0]
    dZ2 = A2 - y  # Gradient lớp đầu ra
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m
    if activation == 'relu':
        dA1 = np.dot(dZ2, W2.T)
        dZ1 = dA1 * (Z1 > 0)  # Đạo hàm ReLU
    else:
        dA1 = np.dot(dZ2, W2.T)
        dZ1 = dA1 * A1 * (1 - A1)  # Đạo hàm Sigmoid
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m
    return dW1, db1, dW2, db2

In [ ]:
def update_weights(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

In [ ]:
def train_model(X_train, y_train, X_test, y_test, hidden_size, activation, batch_size, learning_rate, epochs):
    input_size = X_train.shape[1]  # 784
    output_size = y_train.shape[1]  # 10
    W1, b1, W2, b2 = initialize_weights(input_size, hidden_size, output_size)

    for epoch in range(epochs):
        for i in range(0, X_train.shape[0], batch_size):
            X_batch = X_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            Z1, A1, Z2, A2 = forward_propagation(X_batch, W1, b1, W2, b2, activation)
            dW1, db1, dW2, db2 = backward_propagation(X_batch, y_batch, Z1, A1, Z2, A2, W1, W2, activation)
            W1, b1, W2, b2 = update_weights(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)

        if epoch % 10 == 0:
            _, _, _, A2_train = forward_propagation(X_train, W1, b1, W2, b2, activation)
            train_loss = cross_entropy_loss(y_train, A2_train)
            train_acc = np.mean(np.argmax(A2_train, axis=1) == np.argmax(y_train, axis=1))
            print(f'Epoch {epoch}, Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}')

    return W1, b1, W2, b2

In [ ]:
def evaluate_model(X_test, y_test, W1, b1, W2, b2, activation):
    _, _, _, A2_test = forward_propagation(X_test, W1, b1, W2, b2, activation)
    test_acc = np.mean(np.argmax(A2_test, axis=1) == np.argmax(y_test, axis=1))
    return test_acc

In [ ]:
# Danh sách các bộ siêu tham số
hyperparameters = [
    (32, 0.1, 16, 'relu'),
    (16, 0.01, 64, 'sigmoid'),
    (64, 0.05, 32, 'relu'),
    (32, 0.001, 128, 'sigmoid'),
    (128, 0.1, 64, 'relu')
]

# Thử nghiệm
for hp in hyperparameters:
    batch_size, learning_rate, hidden_size, activation = hp
    accuracies = []
    print(f"\nThử nghiệm với siêu tham số: {hp}")

    for trial in range(5):
        print(f"Lần chạy {trial+1}/5")
        W1, b1, W2, b2 = train_model(x_train, y_train, x_test, y_test, hidden_size, activation,
                                    batch_size, learning_rate, epochs=50)
        acc = evaluate_model(x_test, y_test, W1, b1, W2, b2, activation)
        accuracies.append(acc)
        print(f"Độ chính xác: {acc:.4f}")

    mean_acc = np.mean(accuracies)
    std_acc = np.std(accuracies)
    print(f"Kết quả: Trung bình = {mean_acc:.4f}, Độ lệch chuẩn = {std_acc:.4f}")


Thử nghiệm với siêu tham số: (32, 0.1, 16, 'relu')
Lần chạy 1/5
Epoch 0, Loss: 0.2795, Accuracy: 0.9163
Epoch 10, Loss: 0.1381, Accuracy: 0.9582
Epoch 20, Loss: 0.1183, Accuracy: 0.9638
Epoch 30, Loss: 0.1101, Accuracy: 0.9663
Epoch 40, Loss: 0.1051, Accuracy: 0.9672
Độ chính xác: 0.9481
Lần chạy 2/5
Epoch 0, Loss: 0.2899, Accuracy: 0.9120
Epoch 10, Loss: 0.1420, Accuracy: 0.9558
Epoch 20, Loss: 0.1230, Accuracy: 0.9615
Epoch 30, Loss: 0.1177, Accuracy: 0.9632
Epoch 40, Loss: 0.1131, Accuracy: 0.9640
Độ chính xác: 0.9432
Lần chạy 3/5
Epoch 0, Loss: 0.2800, Accuracy: 0.9155
Epoch 10, Loss: 0.1410, Accuracy: 0.9575
Epoch 20, Loss: 0.1197, Accuracy: 0.9626
Epoch 30, Loss: 0.1109, Accuracy: 0.9656
Epoch 40, Loss: 0.1054, Accuracy: 0.9666
Độ chính xác: 0.9523
Lần chạy 4/5
Epoch 0, Loss: 0.2734, Accuracy: 0.9185
Epoch 10, Loss: 0.1445, Accuracy: 0.9548
Epoch 20, Loss: 0.1329, Accuracy: 0.9588
Epoch 30, Loss: 0.1237, Accuracy: 0.9615
Epoch 40, Loss: 0.1162, Accuracy: 0.9641
Độ chính xác: 0.9